# Crear Pinecone Index - Agenticus

In [ ]:
#@title Haz click aquí -> X y presiona CTRL + ENTER

#Instalar paquetes
!pip install pdfplumber -q
!pip install pinecone-client -q
!pip install langchain -q
!pip install langchain-community -q
!pip install langchain_openai -q

# Importar paquetes
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
import os
from google.colab import drive
from google.colab import userdata

#Setear variables
chunk_size = 1000
chunk_overlap = 200
splits = []
docs_usuario = userdata.get('DOCS_USUARIO')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_ENVIRONMENT'] = 'aws-starter'
embedding_model = OpenAIEmbeddings()

#Montarse a Google Drive
drive.mount('/content/drive')

# Convertir los PDF en un listado de langchain_core documents
for root, dirs, files in os.walk(docs_usuario):
    for file in files:
        #Leer el pdf
        loader = PDFPlumberLoader(os.path.join(root, file))
        #Crear un text_splitter object
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap
        )
        #Load y split el pdf leido, dos pasos en uno
        new_splits = loader.load_and_split(text_splitter)
        #Como son más de un pdf agregamos cada new_split a la lista consolidada splits
        splits = splits + new_splits
        #splits es una lista que contiene varios langchain_core.documents.base.Document

#Crear retriever
vectorstore = Pinecone.from_documents(
    documents=splits, embedding=embedding_model, index_name='asistentecv'
)

print('El Pinecone index ha sido creado')